In [8]:
%matplotlib inline

import matplotlib
from matplotlib import pyplot as plt

import numpy as np
np.random.seed(123)  # for reproducibility

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D, Conv2D
import keras

In [9]:
# design the model

input_shape = (8, 200, 1)
num_classes = 3

model = Sequential()
model.add(Conv2D(32, kernel_size=(2, 10), strides=(1, 1),
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 5), strides=(2, 5)))
model.add(Conv2D(64, (2, 10), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 5)))

model.add(Flatten())
model.add(Dense(500, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

In [10]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.SGD(lr=0.01),
              metrics=['accuracy'])

In [45]:
from influxdb import InfluxDBClient
DB_NAME = 'eeg_test'
DB_HOST = 'localhost'
DB_PORT = 8086
client = InfluxDBClient(host=DB_HOST, port=DB_PORT)
client.create_database(DB_NAME)
client.get_list_database()
client.switch_database(DB_NAME)

In [58]:
alpha = client.query('select * from "autogen"."alpha" GROUP BY "userID" LIMIT 2000')
for r in alpha:
    alpha = r
    break
gamma = client.query('select * from "autogen"."gamma" GROUP BY "userID" LIMIT 2000')
for r in gamma:
    gamma = r
    break
activity = client.query('select * from "autogen"."Activity" GROUP BY "userID" LIMIT 2000')
for r in activity:
    activity = r
    break

In [68]:
resultAll_X = []
for j in range(1):
    result = []
    for i in range(200):
        result.append([alpha[i + (j-1)*200]['Q'], alpha[i + (j-1)*200]['R'], alpha[i + (j-1)*200]['S'], alpha[i + (j-1)*200]['T'], gamma[i+ (j-1)*200]['Q'], gamma[i+ (j-1)*200]['R'], gamma[i + (j-1)*200]['S'], gamma[ i+ (j-1)*200]['T']])
    resultAll_X.append(result)
#resultAll_X[0:3]

resultAll_Y = []
for j in range(1):
    sum_Y = 0
    for i in range(200):
        sum_Y = activity[i + (j-1)*200] + sum_Y
    average = int(sum_Y/200)    
resultAll_Y.append(average)

TypeError: unsupported operand type(s) for +: 'generator' and 'int'

In [65]:
x_train = resultAll_X
y_train = resultAll_Y

model.fit(x_train, y_train,
          batch_size=32,
          epochs=5,
          verbose=2,
          validation_split=0.9,
          callbacks=[history])

ValueError: Error when checking input: expected conv2d_7_input to have 4 dimensions, but got array with shape (200, 8)

In [57]:
class AccuracyHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.acc = []

    def on_epoch_end(self, batch, logs={}):
        self.acc.append(logs.get('acc'))

In [56]:
history = AccuracyHistory()

plt.plot(range(1,11), history.acc)
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.show()

AttributeError: 'AccuracyHistory' object has no attribute 'acc'